In [116]:
#2-step keyword extraction
#Text Mining Domains, VU

import pickle
import glob
import pandas as pd
import json
import os

def new_dict_concat_six (unpickled_dict):
    '''
    takes an unpickled dictionary as input with headline as [1] and list of first five sentences as [2]
    converts to a new dict with same key but value is string of concatenated headline + first five sentences
    filters the values of the keywords
    reurns: new dictionary with keys and concatenated headlines and first_five sentences of those articles which 
        contain the keywords
    '''        
    output = [unpickled_dict[key][1]+ '. ' + ' '.join(unpickled_dict[key][2]) for key in unpickled_dict]
    
    # calling test_dict only gets keys
    new_dict = dict(zip(unpickled_dict, output))
    
    return new_dict

def first_filter(keywords, infolder):
    '''This function will not be used for the end product'''
    for file in glob.glob(infolder):
        #prep filepaths for writing files
        path = os.path.split(file)[0].split('/')[0]
        basename = os.path.basename(file)

        unpickled_dict = pickle.load(open (file, 'rb'))
        #print(unpickled_dict)
        # run the function above to concat headlines and first_five
        new_dict = new_dict_concat_six(unpickled_dict)
        #print(new_dict)
        #print(new_dict)
        #rough filter for only keywords (see above)
        output_dict = {k: v for k, v in new_dict.items() for keyword in keywords if keyword in v}

        #write filtered articles - choosing json this time so I can inspect them in IDE
        if not os.path.exists(f'filteredpickles/{keywords[0]}'):
            os.makedirs(f'filteredpickles/{keywords[0]}')

        with open(f'filteredpickles/{keywords[0]}/{basename.rstrip(".gz.pkl")}.json', 'w', encoding = 'utf-8') as outfile:
            json.dump(output_dict, outfile)
                  
def get_articles(topic):
    '''Download the related articles from the SDG-related folder'''
    df_list = []
    for file in glob.glob(f'filteredpickles/{topic}/*'):
        jsonners = json.load(open(file, 'rb'))
        columns=['text']
        df_list.append(pd.DataFrame.from_dict(jsonners, orient='index', columns= columns))
    big_df = pd.concat(df_list)
    pd.set_option('display.max_colwidth', None)
    print(f'The number of articles left after the first filter is: {len(big_df)}')
    return big_df
                  
def keyword_search(keywords, big_df, treshold = 0):
    '''Loads a (hand-crafted) SDG-specific keyword list.
    Filters all articles containing 'treshold' or more members of the keyword list
    Returns a dataframe containing relevant articles.'''
#phase 2 of keyword search
    article_dictlist = []
    #do a keyword search for the company
    i = 0
    for text in big_df['text']:
        keywords_found = set()
        for keyword in keywords:
            if keyword in text:
                keywords_found.add(keyword)
        if len(keywords_found) >= treshold:
            article_dict = {'identifier' : big_df.index[i], 'text' : text, 'keywords' : keywords_found, 'big_df index' : i}
            article_dictlist.append(article_dict)
        i += 1
    articles = pd.DataFrame(article_dictlist)
    return articles

In [114]:
def two_step_keyword_search(topic, keywords1, infolder, treshold = 3):
    '''Guides the whole two-fold keyword search. Returns a df containing all found articles relevant to the SDG.'''
    
    #first_filter(keywords1, infolder)
    
    #step 1
    big_df = get_articles(topic)
    
    #load keywords for step 2
    with open(f'keywords/{topic}.txt', 'r', encoding = 'utf-8') as infile:
        keywords2 = infile.read().splitlines()
    
    #get the articles
    articles = keyword_search(keywords2, big_df, treshold = treshold)
    print('The number of articles found:', len(articles))

    return articles

## Two-step keyword search

In [125]:
#define the keywords for the 1st rough search
keywords=['poverty', ' aid']

#define topic for folder administration
topic = 'poverty'
#define folder where the articles are located
infolder = 'pickles/*'

articles = two_step_keyword_search(topic, keywords, infolder, treshold = 3)

with open(f'results/keyword_lookup_results_{topic}.csv', 'w') as outfile:
    outfile.write(articles.to_csv())

The number of articles left after the first filter is: 54455
The number of articles found: 1785


## Retrieving Tf-idf scores for keywords

Retrieve random articles to balance tf-idf scores for keywords

In [53]:
def no_filter(infolder):
    for file in glob.glob(infolder):
        #prep filepaths for writing files
        path = os.path.split(file)[0].split('/')[0]
        basename = os.path.basename(file)

        unpickled_dict = pickle.load(open (file, 'rb'))
        #print(unpickled_dict)
        # run the function above to concat headlines and first_five
        new_dict = new_dict_concat_six(unpickled_dict)

        #write filtered articles - choosing json this time so I can inspect them in IDE
        if not os.path.exists(f'filteredpickles/unfiltered'):
            os.makedirs(f'filteredpickles/unfiltered')

        with open(f'filteredpickles/unfiltered/{basename.rstrip(".gz.pkl")}.json', 'w', encoding = 'utf-8') as outfile:
            json.dump(new_dict, outfile)
                  
def random_sample(n):
    df_list = []
    i = 0
    path = "filteredpickles/unfiltered"
    while i < 50:
        filename = random.choice(os.listdir(path))
        jsonners = json.load(open(f'{path}/{filename}', 'rb'))
        df_list.append(pd.DataFrame.from_dict(jsonners, orient = 'index', columns = ['text']))
        i+=1
    random_df = pd.concat(df_list)
    random_sample = random_df.sample(n = n)
    return random_sample

In [54]:
#create json files for all subcorpora, needed for random sampling
#no_filter(infolder)

Compute tf-idf scores for keywords in relevant articles

In [95]:
def compute_tf_idf(articles, keywordlist):
    random_df = random_sample(30000)
    mixed_df = pd.concat([articles,random_df])
    
    stopWords = set(stopwords.words('english'))
    #train and apply tfidf
    #may take a while
    vectorizer = TfidfVectorizer(stop_words = stopWords, ngram_range = (1,1))
    tf_idf_vecs = vectorizer.fit_transform(mixed_df['text'])
    #last step: get tfidf scores for keywords in selected articles    
    doc_scores = []

    keywords_corpus = []
    for keyword in keywordlist:
        if keyword in vectorizer.get_feature_names():
            keywords_corpus.append(keyword)
    print('keywords used:', keywords_corpus)
    
    for i in range(len(articles)):
        df = pd.DataFrame(tf_idf_vecs[i].T.todense(), index=vectorizer.get_feature_names(), columns=["tfidf"])
        df = df.sort_values(by=["tfidf"], ascending=False)
        #print(df)
        doc_score = 0
        keywords_found = set()
        for keyword in keywords_corpus:
            keyword_score = df['tfidf'][keyword]
            if keyword_score > 0:
                keywords_found.add(keyword)
                doc_score += keyword_score
        if len(keywords_found) > 0:
            doc_score = doc_score / len(keywords_found)
        doc_scores.append(doc_score)
        
    scores_df = pd.DataFrame(doc_scores, columns = ['Score'])
    
    articles = pd.concat([articles, scores_df], axis = 1)
    articles = articles.drop('big_df index', axis=1)
    articles = articles.set_index('identifier')
    articles = articles.sort_values(by="Score", ascending = False)
    return articles

In [91]:
compute_tf_idf(articles, keywords2)

keywords used: ['social protection', 'protection systems', 'protection measures', 'vulnerable', 'equal rights', 'economic resources', 'poor', 'ownership', 'control', 'property', 'resilience', 'exposure', 'shocks', 'disasters', 'mobilization', 'development cooperation', 'adequate means', 'policy frameworks', 'poverty eradication', 'human rights', 'working conditions', 'inhuman', 'workers', 'fair wage', 'extortion', 'slavery', 'famine', 'minimum wage', 'child labor', 'hunger', 'entrepreneur', 'free education', 'low wage', 'development', 'social security', 'natural disaster']


text  \
identifier                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              